## Setup

In [1]:
# Connect using pyodbc, sqlalchemy, and pandas
import sqlalchemy
import pandas as pd

server = "sqlsvr-0092-mdp-02.85f8a2f57eaf.database.windows.net"
database = "GlobalMarketing_DW"
username = "pisrc-inkoo"
with open("./db-pass") as f:
    password = f.read()
driver = "ODBC Driver 17 for SQL Server"

engine = sqlalchemy.create_engine(
    f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver.replace(' ', '+')}"
)


def query_db(query, params=None):
    return pd.read_sql(query, engine, params=params)


df = query_db("SELECT @@version;")
print(df[""][0])


Microsoft SQL Azure (RTM) - 12.0.2000.8 
	Jan 26 2022 22:23:07 
	Copyright (C) 2019 Microsoft Corporation



# `bi_dds.DimCustomerMaster`

In [2]:
# Number of entries
query_db(
    """
    SELECT COUNT(*)
    FROM bi_dds.DimCustomerMaster;
    """
)


,
0,2289654


In [3]:
# Number of distinct entries
query_db(
    """
    SELECT COUNT(DISTINCT CustomerMasterKey)
    FROM bi_dds.DimCustomerMaster;
    """
)


,
0,2289654


In [23]:
# Columns of interest
query_db(
    """
    SELECT TOP 10
        CustomerMasterKey,
        AccountName,
        State,
        Office,
        Region,
        Country,
        Status,
        Industry1,
        Industry2,
        CustomerType,
        GlobalUltimateDUNSNumber,
        OEMSegment,
        CreatedDate,
        DunsName_HQ
    FROM bi_dds.DimCustomerMaster;
    """
)


,CustomerMasterKey,AccountName,State,Office,Region,Country,Status,Industry1,Industry2,CustomerType,GlobalUltimateDUNSNumber,OEMSegment,CreatedDate,DunsName_HQ
0,-1,?,?,?,?,?,?,?,?,?,?,?,1900-01-01 00:00:00,None
1,2113355,ABC ELECTRICAL SERVICES LLC,IA,Davenport,North America,USA,Inactive,Other,,Contractor,,,2019-08-19 10:42:05,None
2,2113356,Arrendadora Capela S.A. de C.V.,GRO,Mexico City,Latin America,Mexico,Active,Mining,,End User,,,2019-08-19 10:42:08,None
3,2113357,DuPont,GA,Nashville,North America,USA,Active,Chemicals & Plastics,,End User,,,2019-08-19 10:42:25,None
4,2113358,Eagle Claw Midstream,TX,Dallas,North America,USA,Inactive,Unassigned,,,,,2019-08-19 10:42:26,None
5,2113359,Ebf Enterprises Inc,NC,Charlotte,North America,USA,Active,Other,,Contractor,,,2019-08-19 10:42:26,None
6,2113360,EGIL ENG & CO AS,,Denmark-Ballerup,EMEA,Norway,Active,Other,,End User,355793241,,2019-08-19 10:42:27,None
7,2113361,Equipos Y Refacciones R.Y.E. S.A. De C.V,MEX,Mexico City,Latin America,Mexico,Active,Waste Management,,End User,,,2019-08-19 10:42:28,None
8,2113362,Fabricaciones De Inoxidable S.A. de C.V.,CHI,Guadalajara,Latin America,Mexico,Active,Other,,End User,,,2019-08-19 10:42:29,None
9,2113363,Farmer S.A,Itapua,777,,Paraguay,Inactive,Unassigned,,,,,2019-08-19 10:42:30,None


# `bi_dds.FactPOS`

In [5]:
# Number of entries
query_db(
    """
    SELECT COUNT(*)
    FROM bi_dds.FactPOS;
    """
)


,
0,70295800


In [6]:
# Number of distinct InfluenceBySoldToPartyKey
query_db(
    """
    SELECT COUNT(DISTINCT InfluenceBySoldToPartyKey)
    FROM bi_dds.FactPOS;
    """
)


,
0,397496


In [7]:
# All InfluenceBySoldToPartyKey values map to CustomerMasterKey values
query_db(
    """
    SELECT COUNT(*)
    FROM bi_dds.FactPOS
    WHERE InfluenceBySoldToPartyKey NOT IN (
        SELECT DISTINCT CustomerMasterKey
        FROM bi_dds.DimCustomerMaster
    );
    """
)


,
0,0


In [8]:
# Number of distinct customers with 2021 POS entries
query_db(
    """
    SELECT COUNT(DISTINCT InfluenceBySoldToPartyKey)
    FROM bi_dds.FactPOS
    WHERE InvoiceDateKey >= 20210101
        AND InvoiceDateKey < 20220101;
    """
)


,
0,146892


In [9]:
# Columns of interest
query_db(
    """
    SELECT TOP 10
        InfluenceBySoldToPartyKey,
        SUM(MInvoiceDateExtendedCostAmount) AS sum_MInvoiceDateExtendedCostAmount,
        SUM(PInvoiceDateExtendedCostAmount) AS sum_PInvoiceDateExtendedCostAmount,
        SUM(PLoadDateExtendedCostAmount) AS sum_PLoadDateExtendedCostAmount,
        SUM(QuantityNumber) AS sum_QuantityNumber
    FROM bi_dds.FactPOS
    WHERE InvoiceDateKey >= 20210101
        AND InvoiceDateKey < 20220101
    GROUP BY InfluenceBySoldToPartyKey;
    """
)


,InfluenceBySoldToPartyKey,sum_MInvoiceDateExtendedCostAmount,sum_PInvoiceDateExtendedCostAmount,sum_PLoadDateExtendedCostAmount,sum_QuantityNumber
0,-1,2.668701e+07,2.683206e+07,2.683045e+07,1212443.0
1,2113359,2.381094e+04,2.381094e+04,2.381094e+04,595.0
2,2113360,4.633502e+03,4.521073e+03,4.521073e+03,272.0
3,2113374,6.137360e+03,6.137360e+03,6.137360e+03,235.0
4,2113376,7.834045e+04,7.834045e+04,7.834045e+04,484.0
5,2113381,1.730970e+03,1.730970e+03,1.730970e+03,31.0
6,2113389,7.390530e+03,7.390530e+03,7.390530e+03,58.0
7,2113392,4.308009e+04,4.308009e+04,4.308009e+04,75.0
8,2113395,3.033993e+04,3.033993e+04,3.033993e+04,117.0
9,2113398,1.153141e+03,1.116516e+03,1.116516e+03,24.0


# `bi_dds.FactInvoice`

In [10]:
# Number of entries
query_db(
    """
    SELECT COUNT(*)
    FROM bi_dds.FactInvoice;
    """
)


,
0,14287978


In [11]:
# Number of distinct SoldToPartyKey
query_db(
    """
    SELECT COUNT(DISTINCT SoldToPartyKey)
    FROM bi_dds.FactInvoice;
    """
)


,
0,15146


In [12]:
# All SoldToPartyKey values map to CustomerMasterKey values
query_db(
    """
    SELECT COUNT(*)
    FROM bi_dds.FactInvoice
    WHERE SoldToPartyKey NOT IN (
        SELECT DISTINCT CustomerMasterKey
        FROM bi_dds.DimCustomerMaster
    );
    """
)


,
0,0


In [13]:
# Number of distinct customers with 2021 invoice entries
query_db(
    """
    SELECT COUNT(DISTINCT SoldToPartyKey)
    FROM bi_dds.FactInvoice
    WHERE InvoiceDateKey >= 20210101
        AND InvoiceDateKey < 20220101;
    """
)


,
0,11222


In [14]:
# Columns of interest
query_db(
    """
    SELECT TOP 10
        SoldToPartyKey,
        SUM(ReportingBillingAmount) AS sum_ReportingBillingAmount,
        SUM(Quantity) AS sum_Quantity
    FROM bi_dds.FactInvoice
    WHERE InvoiceDateKey >= 20210101
        AND InvoiceDateKey < 20220101
    GROUP BY SoldToPartyKey;
    """
)


,SoldToPartyKey,sum_ReportingBillingAmount,sum_Quantity
0,-1,8406456.45,3888.85
1,2114494,118019.42,41.00
2,2115679,1277.64,3.00
3,2116315,9522.68,14.00
4,2116398,95647.92,2.00
5,2116464,83652.30,8.00
6,2117051,77550.00,1.00
7,2117817,3565.65,2.00
8,2117851,0.00,1.00
9,2118021,3211.54,2.00


# Joins

In [24]:
# Include all customers that had a POS or invoice entry in 2021
df = query_db(
    """
    WITH
        -- sum of POS amounts in 2021
        pos_query
        AS
        (
            SELECT
                InfluenceBySoldToPartyKey,
                SUM(MInvoiceDateExtendedCostAmount) AS sum_POS_MInvoiceDateExtendedCostAmount,
                SUM(PInvoiceDateExtendedCostAmount) AS sum_POS_PInvoiceDateExtendedCostAmount,
                SUM(PLoadDateExtendedCostAmount) AS sum_POS_PLoadDateExtendedCostAmount,
                SUM(QuantityNumber) AS sum_POS_QuantityNumber
            FROM GlobalMarketing_DW.bi_dds.FactPOS
            WHERE InvoiceDateKey >= 20210101
                AND InvoiceDateKey < 20220101
            GROUP BY InfluenceBySoldToPartyKey
        ),
        -- sum of invoice amounts in 2021
        invoice_query
        AS
        (
            SELECT
                SoldToPartyKey,
                SUM(ReportingBillingAmount) AS sum_Invoice_ReportingBillingAmount,
                SUM(Quantity) AS sum_Invoice_Quantity
            FROM GlobalMarketing_DW.bi_dds.FactInvoice
            WHERE InvoiceDateKey >= 20210101
                AND InvoiceDateKey < 20220101
            GROUP BY SoldToPartyKey
        ),
        -- all SoldToPartyKeys that have either POS or invoice amounts in 2021
        pos_invoice_join
        AS
        (
            SELECT
                COALESCE(p.InfluenceBySoldToPartyKey, i.SoldToPartyKey) AS SoldToPartyKey,
                p.sum_POS_MInvoiceDateExtendedCostAmount,
                p.sum_POS_PInvoiceDateExtendedCostAmount,
                p.sum_POS_PLoadDateExtendedCostAmount,
                p.sum_POS_QuantityNumber,
                i.sum_Invoice_ReportingBillingAmount,
                i.sum_Invoice_Quantity
            FROM pos_query AS p
                FULL OUTER JOIN invoice_query AS i
                ON p.InfluenceBySoldToPartyKey = i.SoldToPartyKey
        )

    SELECT
        c.CustomerMasterKey,
        c.AccountName,
        c.State,
        c.Office,
        c.Region,
        c.Country,
        c.Status,
        c.Industry1,
        c.Industry2,
        c.CustomerType,
        c.GlobalUltimateDUNSNumber,
        c.OEMSegment,
        c.CreatedDate,
        c.DunsName_HQ,
        j.sum_POS_MInvoiceDateExtendedCostAmount,
        j.sum_POS_PInvoiceDateExtendedCostAmount,
        j.sum_POS_PLoadDateExtendedCostAmount,
        j.sum_POS_QuantityNumber,
        j.sum_Invoice_ReportingBillingAmount,
        j.sum_Invoice_Quantity
    FROM
        GlobalMarketing_DW.bi_dds.DimCustomerMaster AS c,
        pos_invoice_join AS j
    WHERE c.CustomerMasterKey = j.SoldToPartyKey;
    """
)
df.to_csv("customers_02.csv")
df


,CustomerMasterKey,AccountName,State,Office,Region,Country,Status,Industry1,Industry2,CustomerType,GlobalUltimateDUNSNumber,OEMSegment,CreatedDate,DunsName_HQ,sum_POS_MInvoiceDateExtendedCostAmount,sum_POS_PInvoiceDateExtendedCostAmount,sum_POS_PLoadDateExtendedCostAmount,sum_POS_QuantityNumber,sum_Invoice_ReportingBillingAmount,sum_Invoice_Quantity
0,2428126,GEORG FISCHER DISA INC,OK,Dallas,North America,USA,Active,Automotive,,OEM,352579336,Manufacturing/Assembly,2017-07-14 23:36:32,None,169524.7100,169524.7100,169524.7100,5900.0,NaN,NaN
1,4261752,Zhengzhou huakong intelligent technology,Henan,Zhengzhou,Asia Pacific,China,Active,Water / Wastewater,,System Integrator,,,2021-04-20 20:10:27,None,73838.1379,74419.6069,74419.6069,406.0,NaN,NaN
2,2327482,Cold Spring Brewing Co,MN,Minn.-St. Paul,North America,USA,Active,Household & P. Care,,End User,,,2017-01-17 09:07:49,None,96661.5600,96661.5600,96661.5600,356.0,NaN,NaN
3,2842380,AUDUBON ENGINEERING CO LLC,LA,Houston,North America,USA,Active,Oil & Gas,,"A&E, EPC, Consultant",080860020,,2017-06-28 20:34:14,None,208268.0400,208268.0400,208268.0400,116.0,NaN,NaN
4,4259044,Terminal Maritima de Carbon S.A. de C.V.,MCH,Queretaro,Latin America,Mexico,Active,Mining,,End User,,,2021-04-13 08:03:13,None,3655.3900,3655.3900,3655.3900,15.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153761,2321391,REYNOLDS CO,LA,Houston,North America,USA,Active,Rockwell Distributor,,RA AUthorized Distr,006958474,,2017-01-17 09:01:23,None,NaN,NaN,NaN,NaN,6507329.69,737808.55
153762,3322940,BARPLATS MINES LTD,NW,South Africa S Off,EMEA,South Africa,Active,Mining,,End User,,,2017-05-06 05:58:27,None,NaN,NaN,NaN,NaN,16260.96,5.00
153763,2935045,AMS SRL,,Italy Sales Office,EMEA,Italy,Active,Life Sciences,,OEM,458514390,Packaging,2017-10-07 03:29:46,None,NaN,NaN,NaN,NaN,89778.34,158.00
153764,3158126,CHINA TOBACCO JIANGXI,Jiangxi,Wuhan,Asia Pacific,China,Active,Food & Beverage,,End User,544706328,,2017-09-30 07:55:33,None,NaN,NaN,NaN,NaN,411999.93,2.25


In [25]:
# Check entries for specific account name
def query_account_name(account_name):
    return query_db(
        """
        WITH
            -- sum of POS amounts in 2021
            pos_query
            AS
            (
                SELECT
                    InfluenceBySoldToPartyKey,
                    SUM(MInvoiceDateExtendedCostAmount) AS sum_POS_MInvoiceDateExtendedCostAmount,
                    SUM(PInvoiceDateExtendedCostAmount) AS sum_POS_PInvoiceDateExtendedCostAmount,
                    SUM(PLoadDateExtendedCostAmount) AS sum_POS_PLoadDateExtendedCostAmount,
                    SUM(QuantityNumber) AS sum_POS_QuantityNumber
                FROM GlobalMarketing_DW.bi_dds.FactPOS
                WHERE InvoiceDateKey >= 20210101
                    AND InvoiceDateKey < 20220101
                GROUP BY InfluenceBySoldToPartyKey
            ),
            -- sum of invoice amounts in 2021
            invoice_query
            AS
            (
                SELECT
                    SoldToPartyKey,
                    SUM(ReportingBillingAmount) AS sum_Invoice_ReportingBillingAmount,
                    SUM(Quantity) AS sum_Invoice_Quantity
                FROM GlobalMarketing_DW.bi_dds.FactInvoice
                WHERE InvoiceDateKey >= 20210101
                    AND InvoiceDateKey < 20220101
                GROUP BY SoldToPartyKey
            ),
            -- all SoldToPartyKeys that have either POS or invoice amounts in 2021
            pos_invoice_join
            AS
            (
                SELECT
                    COALESCE(p.InfluenceBySoldToPartyKey, i.SoldToPartyKey) AS SoldToPartyKey,
                    p.sum_POS_MInvoiceDateExtendedCostAmount,
                    p.sum_POS_PInvoiceDateExtendedCostAmount,
                    p.sum_POS_PLoadDateExtendedCostAmount,
                    p.sum_POS_QuantityNumber,
                    i.sum_Invoice_ReportingBillingAmount,
                    i.sum_Invoice_Quantity
                FROM pos_query AS p
                    FULL OUTER JOIN invoice_query AS i
                    ON p.InfluenceBySoldToPartyKey = i.SoldToPartyKey
            )

        SELECT
            c.CustomerMasterKey,
            c.AccountName,
            c.State,
            c.Office,
            c.Region,
            c.Country,
            c.Status,
            c.Industry1,
            c.Industry2,
            c.CustomerType,
            c.GlobalUltimateDUNSNumber,
            c.OEMSegment,
            c.CreatedDate,
            c.DunsName_HQ,
            j.sum_POS_MInvoiceDateExtendedCostAmount,
            j.sum_POS_PInvoiceDateExtendedCostAmount,
            j.sum_POS_PLoadDateExtendedCostAmount,
            j.sum_POS_QuantityNumber,
            j.sum_Invoice_ReportingBillingAmount,
            j.sum_Invoice_Quantity
        FROM
            GlobalMarketing_DW.bi_dds.DimCustomerMaster AS c,
            pos_invoice_join AS j
        WHERE c.CustomerMasterKey = j.SoldToPartyKey
            AND c.AccountName = ?;
        """,
        [account_name],
    )


In [26]:
query_account_name("Global Process Automation")


,CustomerMasterKey,AccountName,State,Office,Region,Country,Status,Industry1,Industry2,CustomerType,GlobalUltimateDUNSNumber,OEMSegment,CreatedDate,DunsName_HQ,sum_POS_MInvoiceDateExtendedCostAmount,sum_POS_PInvoiceDateExtendedCostAmount,sum_POS_PLoadDateExtendedCostAmount,sum_POS_QuantityNumber,sum_Invoice_ReportingBillingAmount,sum_Invoice_Quantity
0,3387182,Global Process Automation,NC,Charlotte,North America,USA,Active,Pulp & Paper,Life Sciences,System Integrator,,,2017-01-17 09:34:07,None,158692.60,158692.60,158692.60,1048.0,None,None
1,2184278,Global Process Automation,TN,Nashville,North America,USA,Active,Other,,System Integrator,,,2019-05-31 05:17:05,None,17865.08,17865.08,17865.08,4.0,None,None
2,2142956,Global Process Automation,VA,Charlotte,North America,USA,Active,Unassigned,,System Integrator,,,2018-10-31 04:58:36,None,1186.40,1186.40,1186.40,0.0,None,None
3,2128156,Global Process Automation,WA,Seattle,North America,USA,Active,Food & Beverage,Life Sciences,System Integrator,,,2019-10-05 03:04:09,None,6648.24,6648.24,6648.24,5.0,None,None


In [27]:
query_account_name("NHP Electrical Engineering Products Pty")


,CustomerMasterKey,AccountName,State,Office,Region,Country,Status,Industry1,Industry2,CustomerType,GlobalUltimateDUNSNumber,OEMSegment,CreatedDate,DunsName_HQ,sum_POS_MInvoiceDateExtendedCostAmount,sum_POS_PInvoiceDateExtendedCostAmount,sum_POS_PLoadDateExtendedCostAmount,sum_POS_QuantityNumber,sum_Invoice_ReportingBillingAmount,sum_Invoice_Quantity
0,2131612,NHP Electrical Engineering Products Pty,VIC,Victoria/Tasmania,Asia Pacific,Australia,Active,Other,,End User,748231735,,2018-10-09 17:07:40,None,115868.7892,113083.1738,113083.1738,332.0,None,None
1,2121838,NHP Electrical Engineering Products Pty,QLD,Queensland,Asia Pacific,Australia,Active,Other,,End User,000000000,,2019-09-23 16:26:50,None,98465.8865,96606.8816,96606.8816,147.0,None,None
2,2149989,NHP Electrical Engineering Products Pty,AKL,New Zealand,Asia Pacific,New Zealand,Active,Other,,End User,000000000,,2018-10-09 09:54:45,None,222928.7314,206669.8773,206669.8773,918.0,None,None
3,2134958,NHP Electrical Engineering Products Pty,SA,South Aust NorthTerr,Asia Pacific,Australia,Active,Other,,End User,754033702,,2018-09-19 18:35:02,None,34552.7562,33162.1454,33162.1454,29.0,None,None
4,2131608,NHP Electrical Engineering Products Pty,WA,Western Australia,Asia Pacific,Australia,Active,Other,,End User,,,2018-10-09 09:54:50,None,71411.7925,70345.4730,70345.4730,166.0,None,None


In [28]:
query_account_name("Heavy Mineral Developments Pty Ltd")

,CustomerMasterKey,AccountName,State,Office,Region,Country,Status,Industry1,Industry2,CustomerType,GlobalUltimateDUNSNumber,OEMSegment,CreatedDate,DunsName_HQ,sum_POS_MInvoiceDateExtendedCostAmount,sum_POS_PInvoiceDateExtendedCostAmount,sum_POS_PLoadDateExtendedCostAmount,sum_POS_QuantityNumber,sum_Invoice_ReportingBillingAmount,sum_Invoice_Quantity
0,3146780,Heavy Mineral Developments Pty Ltd,QLD,Queensland,Asia Pacific,Australia,Active,Mining,,OEM,000000000,Process,2017-09-22 02:52:37,None,1012.8888,1030.08,1030.08,2.0,None,None
1,2142437,Heavy Mineral Developments Pty Ltd,NSW,New South Wales,Asia Pacific,Australia,Active,Mining,,OEM,,Process,2018-09-28 12:49:58,None,527.7768,515.04,515.04,1.0,None,None
